#Food Recommation System

##install libs..

In [1]:
!pip install biopython matplotlib
from Bio import Entrez, pairwise2
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Seq import Seq
import matplotlib.pyplot as plt

Entrez.email = "your_email@example.com"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.6 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


#create Dummy database Data

In [2]:
#Maps genes to specific nutrient adjustment factors based on genetic variants
GENE_NUTRIENT_MAP = {
    'MTHFR': {'folate': 1.5, 'vitamin_b12': 1.3, 'vitamin_b6': 1.1},
    'VDR': {'vitamin_d': 1.4, 'calcium': 1.2},
    'APOE': {'fat': 0.9, 'omega3': 1.5, 'cholesterol': 0.8},
    'FADS1': {'omega3': 1.3, 'omega6': 0.8},
    'FTO': {'protein': 1.1, 'carbs': 0.9, 'fiber': 1.0},
    'TCF7L2': {'carbs': 0.85, 'fiber': 1.2, 'magnesium': 1.0},
    'PPAR-gamma': {'fat': 0.9, 'fiber': 1.0},
    'GSTM1': {'vitamin_c': 1.2, 'vitamin_e': 1.2, 'cruciferous': 1.3},
    'GSTP1': {'vitamin_c': 1.1, 'selenium': 1.2},
    'COMT': {'vitamin_b6': 1.2, 'magnesium': 1.2, 'green_tea': 1.1},
    'SOD2': {'manganese': 1.3, 'antioxidants': 1.2},
    'NQO1': {'coenzyme_q10': 1.2, 'vitamin_c': 1.1},
    'TNF': {'omega3': 1.0, 'vitamin_d': 1.1},
    'IL6': {'vitamin_d': 1.2, 'zinc': 1.1},
    'CLOCK': {'melatonin': 1.2, 'magnesium': 1.0},
}
#Maps specific SNPs to genes and their functional effects
SNP_EFFECTS = {
    'rs1801133': {'gene': 'MTHFR', 'effect': 'reduced_function'},
    'rs1544410': {'gene': 'VDR', 'effect': 'altered_function'},
    'rs429358': {'gene': 'APOE', 'effect': 'e4_allele'},
    'rs174537': {'gene': 'FADS1', 'effect': 'reduced_conversion'},
    'rs9939609': {'gene': 'FTO', 'effect': 'increased_risk'},
    'rs7903146': {'gene': 'TCF7L2', 'effect': 'increased_risk'},
    'rs1801282': {'gene': 'PPAR-gamma', 'effect': 'altered_function'},
    'rs1695': {'gene': 'GSTP1', 'effect': 'reduced_function'},
    'rs4680': {'gene': 'COMT', 'effect': 'slow_metabolism'},
    'rs4880': {'gene': 'SOD2', 'effect': 'oxidative_stress_risk'},
    'rs1800566': {'gene': 'NQO1', 'effect': 'detox_impairment'},
    'rs1800629': {'gene': 'TNF', 'effect': 'inflammatory_response'},
    'rs1800795': {'gene': 'IL6', 'effect': 'high_inflammation'},
    'rs4580704': {'gene': 'CLOCK', 'effect': 'circadian_disruption'},
}
#Reference Nutrient Values
#FDA Recommended Daily Values
BASE_RDI = {
    'protein': 0.8, 'carbs': 130, 'fat': 0.25, 'fiber': 14, 'calcium': 1000,
    'iron': 8, 'vitamin_c': 75, 'vitamin_d': 15, 'vitamin_b12': 2.4,
    'folate': 400, 'omega3': 1.1, 'omega6': 12, 'magnesium': 400, 'zinc': 11
}
# Convert BMR to TDEE based on activity level
ACTIVITY_MULTIPLIERS = {
    'sedentary': 1.2, 'light': 1.375, 'moderate': 1.55, 'active': 1.725, 'very active': 1.9
}
#Adjust nutrient needs based on existing health conditions
CONDITION_ADJUSTMENTS = {
    'diabetes': {'carbs': 0.8, 'fiber': 1.2},
    'hypertension': {'sodium': 0.6},
    'high_cholesterol': {'fat': 0.8, 'fiber': 1.2},
    'osteoporosis': {'calcium': 1.5, 'vitamin_d': 1.5},
    'anemia': {'iron': 1.8, 'vitamin_b12': 1.5, 'folate': 1.5},
    'celiac': {'iron': 1.3, 'calcium': 1.3, 'vitamin_d': 1.3}
}
FOOD_DATABASE = {
    'chicken_breast': {'protein': 31, 'fat': 3.6, 'carbs': 0, 'fiber': 0, 'iron': 1.0, 'calories': 165},
    'salmon': {'protein': 22, 'fat': 13, 'carbs': 0, 'fiber': 0, 'omega3': 2.3, 'vitamin_d': 11, 'calories': 206},
    'brown_rice': {'protein': 5, 'fat': 1.8, 'carbs': 45, 'fiber': 3.5, 'magnesium': 44, 'calories': 216},
    'sweet_potato': {'protein': 2, 'fat': 0.1, 'carbs': 20, 'fiber': 3, 'vitamin_a': 709, 'calories': 86},
    'broccoli': {'protein': 2.8, 'fat': 0.4, 'carbs': 7, 'fiber': 2.6, 'vitamin_c': 89, 'folate': 63, 'calories': 34},
    'spinach': {'protein': 2.9, 'fat': 0.4, 'carbs': 3.6, 'fiber': 2.2, 'iron': 2.7, 'folate': 194, 'calories': 23},
    'almonds': {'protein': 21, 'fat': 49, 'carbs': 22, 'fiber': 12.5, 'vitamin_e': 25.6, 'magnesium': 270, 'calories': 579},
    'greek_yogurt': {'protein': 10, 'fat': 0.4, 'carbs': 3.6, 'calcium': 115, 'calories': 59},
    'eggs': {'protein': 12.6, 'fat': 10.6, 'carbs': 0.6, 'vitamin_d': 2.0, 'vitamin_b12': 1.1, 'calories': 143},
    'oatmeal': {'protein': 13, 'fat': 6.9, 'carbs': 68, 'fiber': 10.6, 'magnesium': 177, 'calories': 389},
    'lentils': {'protein': 9, 'fat': 0.4, 'carbs': 20, 'fiber': 7.9, 'iron': 3.3, 'folate': 181, 'calories': 116},
    'quinoa': {'protein': 4.4, 'fat': 1.9, 'carbs': 21.3, 'fiber': 2.8, 'magnesium': 64, 'iron': 1.5, 'calories': 120},
     'avocado': {'protein': 2, 'fat': 15, 'carbs': 9, 'fiber': 7, 'vitamin_e': 2.1, 'potassium': 485, 'calories': 160},
    'tofu': {'protein': 8, 'fat': 4.8, 'carbs': 1.9, 'iron': 1.6, 'calcium': 253, 'calories': 76},
    'banana': {'protein': 1.3, 'fat': 0.3, 'carbs': 27, 'fiber': 3.1, 'vitamin_b6': 0.4, 'potassium': 422, 'calories': 105},
    'carrots': {'protein': 0.9, 'fat': 0.2, 'carbs': 10, 'fiber': 2.8, 'vitamin_a': 835, 'vitamin_k': 13, 'calories': 41},
    'mushrooms': {'protein': 3.1, 'fat': 0.3, 'carbs': 3.3, 'fiber': 1, 'vitamin_d': 0.2, 'calories': 22},
    'milk': {'protein': 3.4, 'fat': 1, 'carbs': 5, 'calcium': 120, 'vitamin_b12': 0.9, 'calories': 42},
    'chia_seeds': {'protein': 17, 'fat': 31, 'carbs': 42, 'fiber': 34, 'omega3': 17.8, 'calories': 486},
    'walnuts': {'protein': 15, 'fat': 65, 'carbs': 14, 'fiber': 7, 'omega3': 9.1, 'calories': 654},
    'blueberries': {'protein': 0.7, 'fat': 0.3, 'carbs': 14, 'fiber': 2.4, 'vitamin_c': 9.7, 'calories': 57},
    'beef': {'protein': 26, 'fat': 15, 'carbs': 0, 'iron': 2.6, 'zinc': 4.8, 'calories': 250},
    'pumpkin_seeds': {'protein': 19, 'fat': 49, 'carbs': 54, 'fiber': 6, 'magnesium': 262, 'zinc': 7.8, 'calories': 559},
    'barley': {'protein': 12, 'fat': 2.3, 'carbs': 73, 'fiber': 17.3, 'iron': 3.6, 'magnesium': 133, 'calories': 354},
    'millet': {'protein': 11, 'fat': 4.2, 'carbs': 73, 'fiber': 8.5, 'iron': 3, 'magnesium': 114, 'calories': 378},
    'amaranth': {'protein': 14, 'fat': 7, 'carbs': 65, 'fiber': 6.7, 'iron': 7.6, 'calories': 371},

    # Vegetables
    'kale': {'protein': 4.3, 'fat': 0.9, 'carbs': 9, 'fiber': 4.1, 'vitamin_k': 817, 'vitamin_a': 681, 'calories': 49},
    'cabbage': {'protein': 1.3, 'fat': 0.1, 'carbs': 6, 'fiber': 2.5, 'vitamin_c': 36.6, 'calories': 25},
    'cauliflower': {'protein': 1.9, 'fat': 0.3, 'carbs': 5, 'fiber': 2, 'vitamin_c': 48, 'calories': 25},
    'zucchini': {'protein': 1.2, 'fat': 0.3, 'carbs': 3.1, 'fiber': 1, 'vitamin_c': 17.9, 'calories': 17},

    # Fruits
    'apple': {'protein': 0.3, 'fat': 0.2, 'carbs': 14, 'fiber': 2.4, 'vitamin_c': 4.6, 'calories': 52},
    'orange': {'protein': 0.9, 'fat': 0.1, 'carbs': 12, 'fiber': 2.4, 'vitamin_c': 53.2, 'calories': 47},
    'papaya': {'protein': 0.5, 'fat': 0.3, 'carbs': 11, 'fiber': 1.7, 'vitamin_c': 60.9, 'vitamin_a': 950, 'calories': 43},
    'pomegranate': {'protein': 1.7, 'fat': 1.2, 'carbs': 19, 'fiber': 4, 'vitamin_c': 10.2, 'calories': 83},
    'grapes': {'protein': 0.6, 'fat': 0.2, 'carbs': 17, 'fiber': 0.9, 'vitamin_c': 10.8, 'calories': 69},

    # Legumes (More)
    'black_beans': {'protein': 8.9, 'fat': 0.5, 'carbs': 23.7, 'fiber': 8.7, 'iron': 2.1, 'folate': 128, 'calories': 132},
    'chickpeas': {'protein': 8.9, 'fat': 2.6, 'carbs': 27.4, 'fiber': 7.6, 'iron': 2.9, 'folate': 172, 'calories': 164},
    'kidney_beans': {'protein': 8.7, 'fat': 0.5, 'carbs': 22.8, 'fiber': 6.4, 'iron': 2.6, 'calories': 127},

    # Nuts & Seeds (More)
    'cashews': {'protein': 18, 'fat': 44, 'carbs': 30, 'fiber': 3.3, 'iron': 6.7, 'magnesium': 292, 'calories': 553},
    'sunflower_seeds': {'protein': 21, 'fat': 51, 'carbs': 20, 'fiber': 8.6, 'vitamin_e': 35.2, 'calories': 584},
    'flaxseeds': {'protein': 18, 'fat': 42, 'carbs': 29, 'fiber': 27.3, 'omega3': 22.8, 'calories': 534},

    # Animal Products
    'turkey_breast': {'protein': 29, 'fat': 1, 'carbs': 0, 'iron': 1.1, 'calories': 135},
    'lamb': {'protein': 25, 'fat': 20.8, 'carbs': 0, 'iron': 1.9, 'zinc': 5.3, 'calories': 294},
    'shrimp': {'protein': 20, 'fat': 0.5, 'carbs': 0.2, 'vitamin_b12': 1.1, 'selenium': 39.6, 'calories': 99},

    # Dairy (More)
    'cheddar_cheese': {'protein': 25, 'fat': 33, 'carbs': 1.3, 'calcium': 721, 'vitamin_b12': 0.8, 'calories': 403},
    'buttermilk': {'protein': 3.3, 'fat': 1.7, 'carbs': 4.8, 'calcium': 116, 'calories': 40},
}


#Read the gene info

In [3]:
def fetch_gene_sequence(gene_name):
    try:
        search_handle = Entrez.esearch(db="gene", term=f"{gene_name}[Gene Name] AND homo sapiens[Organism]")
        search_record = Entrez.read(search_handle)
        search_handle.close()
        if not search_record["IdList"]:
            return None
        gene_id = search_record["IdList"][0]
        handle = Entrez.efetch(db="gene", id=gene_id, retmode="xml")
        record = Entrez.read(handle)
        handle.close()
        return record
    except Exception as e:
        print(f"Error fetching gene {gene_name}: {e}")
        return None

def analyze_protein(protein_seq):
    try:
        protein = ProteinAnalysis(str(Seq(protein_seq)))
        return {
            'molecular_weight': protein.molecular_weight(),
            'aromaticity': protein.aromaticity(),
            'instability_index': protein.instability_index(),
            'isoelectric_point': protein.isoelectric_point(),
            'secondary_structure_fraction': protein.secondary_structure_fraction()
        }
    except Exception as e:
        print(f"Error analyzing protein: {e}")
        return None

def analyze_snps(snp_data):
    nutrient_adjustments = {}
    affected_genes = set()
    for snp, genotype in snp_data.items():
        if snp in SNP_EFFECTS:
            gene = SNP_EFFECTS[snp]['gene']
            affected_genes.add(gene)
            if gene in GENE_NUTRIENT_MAP:
                for nutrient, factor in GENE_NUTRIENT_MAP[gene].items():
                    prev = nutrient_adjustments.get(nutrient, 1.0)
                    if abs(factor - 1) > abs(prev - 1):
                        nutrient_adjustments[nutrient] = factor
    return nutrient_adjustments, affected_genes


# Body mass index and Basal Metabolic Rate

##calories needed at rest-bmr

In [4]:
def calculate_bmi(weight, height):
    return weight / ((height / 100) ** 2)

def calculate_bmr(weight, height, age, gender):
    if gender == 'M':
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161


In [5]:


def predict_nutrient_requirements(patient):
    weight, height, age, gender, activity_level, health_conditions, genetic_data = patient.values()
    bmi = calculate_bmi(weight, height)
    bmr = calculate_bmr(weight, height, age, gender)
    tdee = bmr * ACTIVITY_MULTIPLIERS.get(activity_level.lower(), 1.2)#Total Daily Energy Expenditure = BMR × Activity Factor
    requirements = BASE_RDI.copy()
    requirements['protein'] = weight * requirements['protein']

    if gender == 'F':
        if age < 50: requirements['iron'] = 18
        requirements['calcium'] = 1200 if age > 50 else 1000
    if age > 70:
        requirements['vitamin_d'] = 20
        requirements['protein'] = weight * 1.0

    for condition in health_conditions:
        if condition in CONDITION_ADJUSTMENTS:
            for nutrient, factor in CONDITION_ADJUSTMENTS[condition].items():
                if nutrient in requirements:
                    requirements[nutrient] *= factor

    if genetic_data:
        genetic_adjustments, _ = analyze_snps(genetic_data)
        for nutrient, factor in genetic_adjustments.items():
            if nutrient in requirements:
                requirements[nutrient] *= factor

    protein_cal = requirements['protein'] * 4
    fat_cal = tdee * requirements['fat']
    fat_grams = fat_cal / 9
    carb_grams = (tdee - protein_cal - fat_cal) / 4

    requirements['calories'] = tdee
    requirements['carbs'] = carb_grams
    requirements['fat'] = fat_grams

    return {k: round(v) for k, v in requirements.items()}, round(bmi, 1), round(bmr), round(tdee)


In [6]:
def plot_macronutrients(reqs, patient_id="patient"):
    protein_cals = reqs['protein'] * 4
    carb_cals = reqs['carbs'] * 4
    fat_cals = reqs['fat'] * 9

    labels = ['Protein', 'Carbohydrates', 'Fat']
    sizes = [protein_cals, carb_cals, fat_cals]
    colors = ['#ff9999','#66b3ff','#99ff99']

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    plt.title("Macronutrient Distribution")
    plt.savefig(f"{patient_id}_macros.png")
    plt.show()


In [7]:


def recommend_foods(requirements, focus_nutrients=[], top_n=5):
    food_scores = {}
    for food, nutrients in FOOD_DATABASE.items():
        score = 0
        if 'protein' in nutrients: score += nutrients['protein'] / nutrients['calories'] * 100
        if 'fiber' in nutrients: score += nutrients['fiber'] * 2
        for nutrient in focus_nutrients:
            if nutrient in nutrients and nutrients[nutrient] > 0:
                score += 10
        food_scores[food] = score

    sorted_foods = sorted(food_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    recommendations = []
    for food, score in sorted_foods:
        recommendations.append({
            'food': food,
            'score': score,
            'key_nutrients': [n for n in focus_nutrients if n in FOOD_DATABASE[food]],
            'nutrition': FOOD_DATABASE[food]
        })
    return recommendations


In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Output Widget to Display Final Data ---
output = widgets.Output()

# --- Global Variable to Hold Final Patient Data ---
sample_patient = {}

# --- Basic Info Widgets ---
weight = widgets.FloatText(description='Weight (kg):', value=70)
height = widgets.FloatText(description='Height (cm):', value=165)
age = widgets.IntText(description='Age:', value=30)
gender = widgets.Dropdown(options=['M', 'F'], description='Gender:')
activity_level = widgets.Dropdown(options=['low', 'moderate', 'high'], description='Activity:')

# --- Health Conditions Multi-Select ---
health_options = ['diabetes', 'hypertension', 'ldl', 'osteoporosis', 'thyroid']
health_conditions = widgets.SelectMultiple(
    options=health_options,
    description='Health:',
    value=[]
)

# --- SNP Info and Dropdowns ---
rs1801133_info = widgets.HTML("""
<b>rs1801133 (MTHFR gene)</b><br>
Involved in folate metabolism.<br>
<i>CC: Normal, CT: Moderate activity, TT: Low activity</i>
""")
rs1801133 = widgets.Dropdown(options=['Not Sure', 'CC', 'CT', 'TT'], description='rs1801133:')

rs1544410_info = widgets.HTML("""
<b>rs1544410 (VDR gene)</b><br>
Vitamin D receptor function and bone health.<br>
<i>AA: Strong, AG: Moderate, GG: Weaker response</i>
""")
rs1544410 = widgets.Dropdown(options=['Not Sure', 'AA', 'AG', 'GG'], description='rs1544410:')

rs9939609_info = widgets.HTML("""
<b>rs9939609 (FTO gene)</b><br>
Related to obesity and appetite control.<br>
<i>TT: Normal weight tendency, AT: Slight risk, AA: High obesity risk</i>
""")
rs9939609 = widgets.Dropdown(options=['Not Sure', 'TT', 'AT', 'AA'], description='rs9939609:')

rs7903146_info = widgets.HTML("""
<b>rs7903146 (TCF7L2 gene)</b><br>
Influences insulin secretion and diabetes risk.<br>
<i>CC: Normal, CT: Moderate risk, TT: High diabetes risk</i>
""")
rs7903146 = widgets.Dropdown(options=['Not Sure', 'CC', 'CT', 'TT'], description='rs7903146:')

# --- Submit Button ---
submit_btn = widgets.Button(description='Submit', button_style='success')

# --- On Submit Click ---
def on_submit_clicked(b):
    global sample_patient
    sample_patient = {
        "weight": weight.value,
        "height": height.value,
        "age": age.value,
        "gender": gender.value,
        "activity_level": activity_level.value,
        "health_conditions": list(health_conditions.value),
        "genetic_data": {
            'rs1801133': rs1801133.value,
            'rs1544410': rs1544410.value,
            'rs9939609': rs9939609.value,
            'rs7903146': rs7903146.value
        }
    }

    with output:
        clear_output()

        # --- Nutrient Calculation Logic ---
        try:
            # 1. Predict Requirements
            reqs, bmi, bmr, tdee = predict_nutrient_requirements(sample_patient)
            print(f"✅ Patient BMI: {bmi:.2f}")
            print(f"🔥 BMR: {bmr:.2f} kcal/day")
            print(f"⚡ TDEE: {tdee:.2f} kcal/day\n")
            print("🔬 Nutrient Requirements:")
            for k, v in reqs.items():
                print(f"  {k}: {v}")

            # 2. Plot Macronutrients
            plot_macronutrients(reqs, patient_id="sample_patient")

            # 3. Analyze SNPs
            _, genes = analyze_snps(sample_patient["genetic_data"])
            focus_nutrients = []
            for gene in genes:
                if gene in GENE_NUTRIENT_MAP:
                    focus_nutrients.extend(GENE_NUTRIENT_MAP[gene].keys())
            focus_nutrients = list(set(focus_nutrients))

            # 4. Recommend Foods
            recommendations = recommend_foods(reqs, focus_nutrients)

            print("\n🍎 Top Food Recommendations:")
            for r in recommendations:
                print(f"- {r['food'].replace('_', ' ').title()} (Key nutrients: {', '.join(r['key_nutrients'])})")

        except Exception as e:
            print(f"❌ Error: {e}")

submit_btn.on_click(on_submit_clicked)

# --- Display All Widgets in a Clean Layout ---
display(widgets.VBox([
    widgets.HTML("<h3>🧍 Basic Information</h3>"),
    weight, height, age, gender, activity_level,

    widgets.HTML("<h3>🩺 Health Conditions</h3>"),
    health_conditions,

    widgets.HTML("<h3>🧬 Genetic Information</h3>"),
    rs1801133_info, rs1801133,
    rs1544410_info, rs1544410,
    rs9939609_info, rs9939609,
    rs7903146_info, rs7903146,

    submit_btn,
    output
]))
